# Trabalho #3 - Treinamento customizado

Nesse trabalho você vai treinar uma RNA para prever se um tumor é maligno ou benigno usando o conjunto de dados "Breast Cancer Dataset", disponível no UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)


Esse conjunto de dados foi obtido pelo Hospital da University de Wisconsin, Madison por: O. L. Mangasarian e W. H. Wolberge, "Cancer diagnosis via linear programming", SIAM News, Volume 23, Number 5, September 1990, pp 1 & 18.

## Coloque o seu nome:

Nome:

## 1. Importar bibliotecas

Execute a célula abaixo para importar as principais bilbiotecas necessárias.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 2.  Carregar dados

Esse conjunto de dados possui 699 exemplos, sendo que cada exemplo é composto por 10 características ontidas por exames de células de tecidos que podem ser da classe de cancer maligno ou benigino.

As características de cada exemplo são as seguintes:

1. Número de identificação da amostra: id
2. Espessura do aglomerado: 1 - 10
3. Uniformidade do tamanho da célula: 1 - 10
4. Uniformidade da forma celular: 1 - 10
5. Adesão Marginal: 1 - 10
6. Tamanho de célula epitelial única: 1 - 10
7. Núcleos expostos: 1 - 10
8. Cromatina Suave: 1 - 10
9. Nucléos normais: 1 - 10
10. Mitoses: 1 - 10
11. Casse: 2 para benigno e 4 para maligno

Execute a célula abaixo para carregar o conjunto de dados e criar um DataFrame Pandas. Para facilitar o entendimento dos dados vamos definir explicitamente os nomes das colunas porque o arquivo CSV original não possui o cabeçalho com os nomes das colunas.

In [ ]:
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
data_file = tf.keras.utils.get_file("breast_cancer.csv", DATASET_URL)
col_names = ["id", "espessura", "un_tam_cel", "un_forma_cel", "adesao_marginal", "tam_cel_epit", "nucleos_exp", "cromatina", "nucleos_normais", "mitoses", "classe"]
df = pd.read_csv(data_file, names=col_names, header=None)

Execute as duas células abaixo para visualizar os dados e verificar o número de exemplos.

In [ ]:
df.head()

In [ ]:
print('Dimensão dos dados:', df.shape)

## 3. Pré-processamento dos dados

Os dados precisam ser pré-processados para poderem ser utilzados por uma RNA.

As seguintes etapas devem ser realizadas no processamento:

1. Retirar a coluna da identificação da amostra ("id");
2. Eliminar dados que possuem valores "desconhecidos";
3. Transformar classes de índices 2 e 4 para 0 e 1, 0 é benigno e 1 é maligno;
4. Dividir dados nos conjuntos de treinamento e teste;
5. Separar coluna das classes (dados de saídas reais) das outras colunas (dados de entradas);
6. Normalizar os dados de entrada;
7. Conveter DataFrame Pandas para tf.Tensor.

### 3.1 Retirar coluna de identificação da amostra ("id")

Execute a célula baixo para realizar essa operação.

In [ ]:
df.pop("id")
df.head()

### 3.2 Eliminar dados "desconhecidos"

Se você inspecionar os dados vai verificar que existem valores "desconehcidos" na coluna de "nucleos_exp". Para verificar quais amostras possuem valores desconhecidos execute a célula abaixo.

In [ ]:
df[df["nucleos_exp"] == '?']

Deve-se eliminar as linhas que possuem dados desse tipo. Além disso, a coluna "nucleos_exp" não é uma coluna numérica e, portanto, deve ser convertida para valores numéricos. Execute a célula abaixo para realizar essas operações.

In [ ]:
# Elimina linhas com dados desconhecidos na coluna "nucleos_exp"
df = df[df["nucleos_exp"] != '?' ]

# Converte coluna "nucleo_exp" para valores numéricos
df.nucleos_exp = pd.to_numeric(df.nucleos_exp)
df

- Observe que o conjunto de dados agora tem 683 exemplos, ou seja, 16 exemplos foram retirados porque tinham valores "unknown".

### 3.3 Transformar código das classes de câncer

Primeiramente vamos verificar o número de exemplos de cada classe. Lembre que:

- Classe = 2 $\to$ câncer benigno
- Classe = 4 $\to$ câncer maligno

Observa-se que é importante fazer essa verificação porque se o número de exemplos das classes for muito desbalanceado temos que usar técnicas especiais para treinar a RNA, como já visto anteriormente.

Para visualizar o número de exemplos de cada classe vamos calcular e fazer o gráfico do histograma da coluna "classe".

In [ ]:
df['classe'].hist(bins=20)

Para poder modelar esse problema como um problema de classificão binária, que detecta se o tumor é maligno ou não, temos que alterar os códigos das classes para o seguinte:

- Câncer benigno (2.0) = 0
- Câncer maligno (4.0) = 1

In [ ]:
df['classe'] = np.where(df['classe'] == 2, 0, 1)
df

### Exercício #1: Dividir e embaralhar conjunto de dados

Vamos dividir o conjunto de dados em conjuntos de treinamento e teste. Como o número de amostras é pequeno, faremos a validação no conjunto de teste.

Nessa divisão vamos utilizar 80% dos dados como sendo de treinamento e 20% como sendo de teste/validação.

Para realizar essa divisão usaremos a função `train_test_split()` da biblioteca ScikitLearn. Observe que você deve usar essa função também para embaralhar aleatoriamente os dados.  

In [ ]:
# Para você fazer: Dividir e embaralhar dados

# Importa função para dividir conjunto de dados
from sklearn.model_selection import train_test_split

# Realiza divisão dos dados
# Inclua seu código aqui
#


**Saída esperada:**

    Dimensão dos dados de treinamento: (546, 10)
    Dimensão dos dados de teste: (137, 10)

### 3.4 Separar coluna das classes (saída desejada)

Devemos separar a coluna das classes dos conjuntos de treinamento e teste para criar as saídas desejadas de treinamento e teste.

In [ ]:
train_Y = train.pop("classe")
test_Y = test.pop("classe")
train_Y

Vamos calcular as estatísticas básicas das saídas dos conjuntos de treinamento e teste para verificar se ambos possuem a mesma distribuição.

In [ ]:
print('Estatística das saídas de treinamento:\n', train_Y.describe())
print('\nEstatística das saídas de teste:\n', test_Y.describe())

### 3.5 Normalizar dados de entrada

Antes de normalizar os dados de entrada é importante calcular as suas estatísticas básicas. Os valores de média e desvio padrão das caracteríticas dos dados de entrada de treinamento serão usados para normalizar os dados de treinamento e de teste.

In [ ]:
train_stats = train.describe()
train_stats = train_stats.transpose()
print(train_stats.shape)
train_stats

### Exercícios #2: Normalizar dados de entrada

Os dados de entrada serão normalizados para terem média igual a zero e disvio padrão igual a 1. Assim, a normalizaçãp de cada coluna deve ser feita de acordo com a seguinte equação:

$$X_{norm,i} = \frac{(X_i - \mu_i)} {\sigma_i}$$

onde $X_i$ é a i-ésima coluna dos dados, $\mu_i$ é a média da i-ésima coluna, $\sigma_i$ é o desvio padrão da i-ésima coluna e $X_{norm,i}$ é a i-ésima coluna dos dados normalizada.

In [ ]:
# Para você fazer: Normalizar dados

# Define função para normalizar as colunas
# Inclua seu código aqui
#

# Normaliza dados entrada de treinamento e teste
# Inclua seu código aqui
#

# Visualiza dados de entrada de treinamento normalizados
X_train_norm

In [ ]:
X_train_norm.describe().transpose()

**Saída esperada:**

       count	        mean	std  	min  25% 50%	  75%	 max
       espessura	     546.0	  -2.373966e-17  	1.0	  -1.225570   -0.871516  -0.163409  0.544698  1.960912
       un_tam_cel	     546.0	-8.743515e-17	1.0	-0.706795	-0.706795	-0.706795	0.566836	2.158875
       un_forma_cel  	546.0	9.800870e-17	1.0	-0.746659	-0.746659	-0.746659	0.549607	2.169940|    
       adesao_marginal 	546.0	9.190857e-17	1.0	-0.658547	-0.658547	-0.658547	0.372715	2.435239
       tam_cel_epit     546.0	1.352195e-16	1.0	-0.998697	-0.563163	-0.563163	0.307905	2.921108
       nucleos_exp  	    546.0	-4.636096e-17	1.0	-0.695540	-0.695540	-0.695540	0.612569	1.782983
       cromatina        546.0	-3.700743e-17	1.0	-0.982966	-0.583636	-0.184306	0.215024	2.611003
       nucleos_normais   546.0	2.257047e-16	1.0	-0.620388	-0.620388	-0.620388	0.347417	2.283027
       mitoses	         546.0	-9.007853e-17	1.0	-0.352816	-0.352816	-0.352816	-0.352816	4.716589

### Exercício #3: Converter DataFrame para tf.Tensor

Para os dados poderem ser usados por uma RNA em um loop de treinamento customizado eles devem estar no forma de tensores do TensorFlow, assim, vamos converter os dados para tf.Tensor.

In [ ]:
# Converte entradas para tf.Tensor
# Inclua seu código aqui
X_train =
X_test =

# Convert saídas para tf.Tensor e ajusta dimensões
# Inclua seu código aqui
Y_train =
Y_test =

print('Dimensão dos dados de entrada de treinamemto:', X_train.shape)
print('Dimensão dos dados de entrada de teste:', X_test.shape)
print('Dimensão dos dados de saída de treinamemto:', Y_train.shape)
print('Dimensão dos dados de saída de teste:', Y_test.shape)

**Saída esperada:**

    Dimensão dos dados de entrada de treinamemto: (546, 9)
    Dimensão dos dados de entrada de teste: (137, 9)
    Dimensão dos dados de saída de treinamemto: (546, 1)
    Dimensão dos dados de saída de teste: (137, 1)

In [ ]:
print(X_train[:10])
print(Y_train[:10])

**Saída esperada:**

    tf.Tensor(
    [[ 1.9609115   2.158875    1.8458735   0.02896096  1.6145062   0.4060258
      -0.18430609  0.67001873 -0.35281572]
     [-0.51746273 -0.70679533 -0.7466588  -0.658547   -0.56316274 -0.69553983
      -0.98296577 -0.62038773 -0.35281572]
     [-1.2255697  -0.70679533 -0.09852573 -0.658547   -0.56316274 -0.69553983
      -0.98296577 -0.62038773 -0.35281572]
     [-1.2255697  -0.70679533 -0.42259225 -0.658547   -0.56316274 -0.42014843
       0.21502376 -0.29778612 -0.35281572]
     [-0.8715162   0.5668359   1.1977404   1.0602229   0.30790484  1.7829828
       1.4130133   0.99262035 -0.35281572]
     [ 1.9609115   2.158875    1.1977404   1.7477309   1.6145062  -0.69553983
       2.611003    2.2830267   0.77371866]
     [-1.2255697  -0.70679533 -0.7466588  -0.658547   -0.9986965  -0.69553983
      -0.98296577  0.02481551 -0.35281572]
     [ 0.19064417 -0.06997974 -0.42259225  1.7477309   0.74343866  1.7829828
       1.8123431  -0.62038773  0.21045148]
     [ 1.9609115   0.24842808  0.54960734  0.37271494 -0.12762895  0.4060258
       1.4130133   0.02481551 -0.35281572]
     [ 0.19064417 -0.70679533 -0.7466588  -0.658547   -0.56316274 -0.69553983
      -0.18430609 -0.62038773 -0.35281572]], shape=(10, 9), dtype=float32)
    tf.Tensor(
    [[1]
     [0]
     [0]
     [0]
     [1]
     [1]
     [0]
     [1]
     [1]
     [0]], shape=(10, 1), dtype=int32)

## 4. Configuração e compilação da RNA

### Exercício #4: Configuração da RNA

Para realizar essa tarefa de classificação binária vamos utilizar uma RNA com 3 camadas tipo densa. Na célula abaixo configure a sua RNA usando os seguintes parâmetros:

- Primeira camada: 128 neurônios e função de ativação Relu;
- Segunda camada: 64 neurônios e função de ativação Relu;
- Camada de saída: 1 neurônio e função de ativação sigmoide.

In [ ]:
# Para você fazer: Configuração da RNA

# Inclua seu código aqui
#

# Resumo da RNA
rna.summary()

**Saída esperada:**

    Model: "sequential"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    dense (Dense)                (None, 128)               1280      
    _________________________________________________________________
    dense_1 (Dense)              (None, 64)                8256      
    _________________________________________________________________
    dense_2 (Dense)              (None, 1)                 65        
    =================================================================
    Total params: 9,601
    Trainable params: 9,601
    Non-trainable params: 0
    _________________________________________________________________

### Exercício #5: Definição do otimizador da RNA, função de custo e métrica

Na célula abaixo defina o otimizador, a função de custo e a métrica que serão usados no treinamento da RNA.

- Otimizador: Adam com taxa de aprendizado de 0.001;
- Função de custo: BinaryCrossentropy
- Métrica: Accuracy

Observa-se que deve-se usar as versões na forma de classes de todas essas funções.

In [ ]:
# Para você fazer: definir otimizador, função de custo e métrica

# Define objeto otimizador usando tf.keras.optimizer.Addam
# Inclua seu código aqui
#

# Define objeto função de custo usando tf.keras.losses.BinaryCrossentropy
# Inclua seu código aqui
#

# Define objeto métrica usando tf.keras.metrics.BinaryAccuracy
# Inclua seu código aqui
#

Vamos avaliar a RNA não treinada para termos uma base do resultado esperado do treinamento. Execute as células a seguir para realizar essa avaliação.

In [ ]:
# Calcula previsões da RNA não treinada
outputs = rna.predict(X_test)

# Calcula função de custo
loss_value = loss_object(y_true=Y_test, y_pred=outputs)
print("Custo antes do treinamento =", loss_value.numpy())

# Calcula métrica
accuracy = metric_object(y_true=Y_test, y_pred=outputs)
print("Métrica antes do treinamento =", accuracy.numpy())

**Saída esperada:**

    Custo antes do treinamento = 0.766273
    Métrica antes do treinamento = 0.16788322

In [ ]:
# Importa funções para calcula matriz e confusão
from sklearn.metrics import confusion_matrix
import itertools

# Define função para construir matriz de confusão
def plot_confusion_matrix(y_true, y_pred, title='', labels=[0,1]):
    cm = confusion_matrix(y_true, y_pred)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title(title)
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
          plt.text(j, i, format(cm[i, j], fmt),
                  horizontalalignment="center",
                  color="black" if cm[i, j] > thresh else "white")
    plt.show()

plot_confusion_matrix(Y_test, tf.round(outputs), title='Matriz de confusão gerada pela RNA não treinada')

# 5. Treinamento da RNA

Para treinar essa RNA vamos criar um loop customizado usando a função `tf.GradientTape()`.


### Exercício #6: Criar função para calcular gradientes e atualizar parâmetros

Na célula abaixo crie um função que calcula o gradiente da função de custo em relação aos parâmetros da RNA e depois atualiza esses parâmetros usando o otimizador configurado anteriormente.

Para acessar os parâmetros de um modelo do TensorFlow basta usar `model.trainable_weights`.

In [ ]:
# Para você fazer: criar função que calcula gradientes e atualiza parâmetros da RNA
@tf.function()
def apply_gradient(optimizer, loss_object, model, x, y):
    '''
    Função para calcular o gradinet e atualizar os parâmetros da RNA

    Argumentos:
        optimizer: otimizador configurado para atualizar os parâmetros
        loss_object: função de custo configurada anteriormente
        model: RNA que está sendo treinada
        x: tensor com os dados de entrada de treinamento
        y: saídas desejadas dos dados de treinamento

    Retorna:
        logits = saídas previstas pela RNA
        loss_value = valor da função de custo
    '''

    # Inclua seu código aqui
    #

    return logits, loss_value

Execute a célula abaixo para testar a sua função `apply_gradient()`.

In [ ]:
# Cria nova RNA iagual a configurada
test_model = rna

# Calcula saída prevista e função de custo
test_logits, test_loss = apply_gradient(optimizer, loss_object, test_model, X_test, Y_test)

print('Primeiras 5 saídas:', test_logits.numpy()[:5])
print('\nFunção de custo =', test_loss.numpy())

del test_model
del test_logits
del test_loss

**Saída esperada:**

    Primeiras 5 saídas: [[0.57383853]
     [0.53157353]
     [0.43801865]
     [0.47596937]
     [0.4938018 ]]
    
    Função de custo = 0.6647787

### Exercício #7: Cálculo da função de custo e métrica para os dados de validação

No final de cada época de treinamento, temos que validar a RNA no conjunto de dados de teste. Crie uma função que calcula a função de custo e a métrica para os dados validação.

In [ ]:
# Para você fazer: função para calcular custo e métrica dos dados de validação

# Função para calcular custo e métrica dos dados de validação
@tf.function()
def perform_validation(model, x_val, y_val):
    #Calcula custo dos dados de validação
    # Inclua seu código aqui
    #

    # Calcula classes arredondando as saídas previstas (valores iguais a 0 ou 1)
    # Inclua seu código aqui
    #

    # Calcula métrica para dados de validação
    # Inclua seu código aqui
    #

    return val_loss, val_accuracy

Execute a célula abaixo para testar a sua função `perform_validation()`.

In [ ]:
val_loss, val_accuracy = perform_validation(rna, X_test, Y_test)

print('Função de custo para os dados de teste =', val_loss.numpy())
print('Exatidão para os dados de teste =', val_accuracy.numpy())

**Saída esperada:**

    Função de custo para os dados de teste = 0.7662731
    Exatidão para os dados de teste = 0.16788322

### Exercício #8: Loop e treinamento customizado

Usando a função `apply_gradient()` vamos criar um loop de treinamemto customizado. Utilize 1000 épocas de treinamento.


In [ ]:
# Para você fazer: Loop de treinamento customizado

# Define número de épocas
num_epocas = 1000

# Loop de treinamento
for i in range(num_epocas):
    # Calcula gradientes e atualiza parâmetros da RNA
    # Inclua seu código aqui
    #

    # Calcula métrica para dados de treinamento
    # Inclua seu código aqui
    #

    # Calcula função de custo e métrica para dados de validação
    # Inclua seu código aqui
    #

    # Imprime resultado da função de custo e métrica da época
    if i % 100 == 0:
        print('Época:', i, '-', 'custo =', loss_value.numpy(), '-', 'exatidão =', accuracy.numpy(), '-', 'custo_val =', val_loss.numpy(), '-', 'val_exatidão =', val_accuracy.numpy())

# Imprime resultado final
print('\nCusto final =', loss_value.numpy())
print('Exatidão final=', accuracy.numpy())
print('\nCusto final de validação =', val_loss.numpy())
print('Exatidão final de validação =', val_accuracy.numpy())

**Saída esperada:**

    Época: 0 - custo = 0.68436486 - exatidão = 0.34634146 - custo_val = 0.6501663 - val_exatidão = 0.3448276
    Época: 100 - custo = 0.058527242 - exatidão = 0.9515191 - custo_val = 0.080492094 - val_exatidão = 0.9515428
    Época: 200 - custo = 0.03760329 - exatidão = 0.96556544 - custo_val = 0.07935773 - val_exatidão = 0.9655706
    Época: 300 - custo = 0.018621787 - exatidão = 0.9716702 - custo_val = 0.09142268 - val_exatidão = 0.97165996
    Época: 400 - custo = 0.008029577 - exatidão = 0.9759616 - custo_val = 0.11785624 - val_exatidão = 0.97595173
    Época: 500 - custo = 0.0036735435 - exatidão = 0.9789992 - custo_val = 0.14663258 - val_exatidão = 0.9789901
    Época: 600 - custo = 0.0019131048 - exatidão = 0.9810311 - custo_val = 0.17128268 - val_exatidão = 0.98102283
    Época: 700 - custo = 0.0011265007 - exatidão = 0.9824835 - custo_val = 0.19244863 - val_exatidão = 0.982476
    Época: 800 - custo = 0.00072901906 - exatidão = 0.983407 - custo_val = 0.21075746 - val_exatidão = 0.9833984
    Época: 900 - custo = 0.0005036661 - exatidão = 0.984111 - custo_val = 0.2269108 - val_exatidão = 0.98410314
    
    Custo final = 0.0003456268
    Exatidão final= 0.98466927

    Custo final de validação = 0.24358612
    Exatidão final de validação = 0.98466206

## 7. Avaliação e teste da RNA

### Exercício #9: Cálculo da função de custo e métrica para os dados de teste

Na célula abaixo calcule a função de custo e a métrica para os dados de teste.

In [ ]:
#Para você fazer: cálculo da função de custo e métrica para os dados de teste

# Calcula saída prevista para os dados de teste
# Inclua seu código aqui
#

# Calcula função de custo para os dados de teste
# Inclua seu código aqui
#

# Calcula métrica para os dados de teste
# Inclua seu código aqui
#

print("Custo =", loss_value.numpy())
print("Exatidão =", metric_value.numpy())

**Saída esperada:**

    Custo = 0.24358612
    Exatidão = 0.9846549

Execute a célula abaixo para calcular a matriz de confusão para a RNA treinada.

In [ ]:
plot_confusion_matrix(Y_test, tf.round(outputs), title='Matriz de confusão da RNA treinada')

- Observa-se que o resultado da RNA é bastante satisfatório, sendo que apenas algumas amostras são classificadas erradas.

- Cerca de 96% das amostra são classificadas corretamente.